In [1]:
from pathlib import Path
import pandas as pd
import numpy as np
import json
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"


In [8]:
# islands = [  2,  4,  8, 16, 36]
# # equals  [ 72, 36, 18,  9,  4] ranks/island
# migrations_prob = [0.01, 0.10, 0.30, 0.50, 0.70, 0.90, 0.99]
# pollination = [True, False]
# mate_prob = [0.1, 0.325, 0.55, 0.775]
# mut_prob = [0.1, 0.325, 0.55, 0.775]
# rand_prob = [0.1, 0.325, 0.55, 0.775]

def pop_std(x):
    return x.std(ddof=0)

# load all data
limits = {
    "quartic": [0.5, -0.1, 0.5, 10, 100],
    "rastrigin": [1., -0.1, 1.0, 0, 256],
    "birastrigin": [40., -0.1, 50.0, 0, 256],
}
# max_gens = 256
# full_df = []
exps = {
    "quartic": [2, 3, 4, 5],
    "rastrigin": [1, 2, 3, 4, 5],
    "birastrigin": [1, 2, 3, 4, 5],  #
}
exp_dfs = {}
for exp in exps:  #
    full_df = []
    # print(exp)
    # exp = "birastrigin"
    for t in exps[exp]:
        root = Path("logs/search/")
        name = Path("results/overall")
        res_dict = Path(f"results/{exp}_grid_{t}.csv")
        df = pd.read_csv(str(res_dict))

        # best = df[(df["loss1"] <= limits[0]) & (df["loss1"] >= limits[1]) & (df["gen1"] <= max_gens)]
        df["islands"] = df["islands"].replace(32, 36)  # mix up with the amount of islands (shouldnt matter too much)
        # keep all results?? why not :shrug:
        full_df.append(df)
        # if full_df is None:
        #     full_df = df
        #     continue

    # combining the dataframes together
    # need to move through the whole file and check that all the exps are the same parameters
    # TODO: change the 32s to 36s in rastrigin
    full_df = pd.concat(full_df, ignore_index=True)
    # print(full_df[
    #     (full_df["islands"] == 8) & (full_df["migration"] == 0.7) & (full_df["pollination"] == True) &
    #     (full_df["mate_prob"] == 0.55) & (full_df["mut_prob"] == 0.1) & (full_df["rand_prob"] == 0.1)
    # ])
    # # print(full_df["islands"].unique())
    grouped = full_df.groupby(by=["islands", "migration", "pollination", "mate_prob", "mut_prob", "rand_prob"], as_index=False).agg({"avg_loss": ['mean', "min", "max", pop_std], "avg_gen": ['mean', 'min', 'max', 'count']})
    grouped.columns = ["islands", "migration", "pollination", "mate_prob", "mut_prob", "rand_prob", 'lossmean', 'lossmin', 'lossmax', "lossstd", 'genmean', 'genmin', 'genmax', 'counts']
    grouped = grouped.reset_index()
    exp_dfs[exp] = grouped

    # print(grouped.sort_values(by="lossmean")[:15])
    best = grouped[
        (grouped["lossmean"] <= limits[exp][0]) & (grouped["lossmean"] >= limits[exp][1]) & (grouped["lossstd"] <= limits[exp][2]) & (grouped["islands"] <= 40) &
        (grouped["mate_prob"] > 0.5) &
        (grouped["migration"] == 0.7) & (grouped["genmin"] >= limits[exp][3]) & (grouped["genmean"] < limits[exp][4]) & (grouped["islands"] > 4) & (grouped["lossstd"] > 0.)
    ]
    print(exp)
    print(best.sort_values(by=["lossstd", "lossmean"])[:15])  # "lossstd",



quartic
      index  islands  migration  pollination  mate_prob  mut_prob  rand_prob  lossmean   lossmin   lossmax   lossstd    genmean     genmin      genmax  counts
2155   2155        8        0.7         True      0.550     0.550      0.775  0.000795  0.000182  0.001429  0.000525  96.750000  39.000000  204.000000       4
2158   2158        8        0.7         True      0.550     0.775      0.550  0.002209  0.001219  0.002778  0.000590  89.250000  40.000000  138.000000       4
2081   2081        8        0.7        False      0.550     0.100      0.325  0.001345  0.000869  0.002391  0.000611  83.041667  66.250000  114.083333       4
2090   2090        8        0.7        False      0.550     0.550      0.550  0.002382  0.001774  0.003437  0.000662  95.177083  67.291667  121.250000       4
2927   2927       16        0.7         True      0.550     0.775      0.775  0.001721  0.000033  0.002843  0.001040  92.750000  17.000000  144.000000       4
2938   2938       16        0.7       

In [ ]:
# find the parameters which give the best and most consistent loss values across all exps
